In [20]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score,train_test_split, KFold, cross_val_predict
from sklearn.metrics import mean_squared_error,r2_score,roc_curve,auc,precision_recall_curve, accuracy_score, \
recall_score, precision_score, confusion_matrix
from sklearn.tree import DecisionTreeRegressor,DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV, ParameterGrid, StratifiedKFold
from sklearn.ensemble import BaggingRegressor,BaggingClassifier,AdaBoostRegressor,AdaBoostClassifier
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression,LogisticRegression
from sklearn.feature_selection import VarianceThreshold
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import GradientBoostingRegressor
from pyearth import Earth
from sklearn.impute import KNNImputer
import xgboost as xgb
import itertools as it
import time as time 
import warnings
warnings.filterwarnings('ignore')

In [21]:
data = pd.read_csv('salaries.csv')

data.head()

,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,2023,MI,FT,AWS Data Architect,258000,USD,258000,US,100,US,L
1,2023,SE,FT,Data Scientist,225000,USD,225000,US,0,US,M
2,2023,SE,FT,Data Scientist,156400,USD,156400,US,0,US,M
3,2023,SE,FT,Data Engineer,190000,USD,190000,US,100,US,M
4,2023,SE,FT,Data Engineer,150000,USD,150000,US,100,US,M


In [22]:
data.job_title.unique()

array(['AWS Data Architect', 'Data Scientist', 'Data Engineer',
       'Applied Scientist', 'ML Engineer', 'Data Analyst',
       'Machine Learning Engineer', 'Data Manager',
       'Product Data Analyst', 'Autonomous Vehicle Operations Lead',
       'Applied Machine Learning Engineer', 'Analytics Engineer',
       'ETL Developer', 'Business Intelligence Engineer',
       'BI Data Analyst', 'Machine Learning Developer',
       'Research Engineer', 'Data Integration Specialist',
       'Data Architect', 'BI Developer', 'Data Visualization Analyst',
       'Head of Data', 'Autonomous Vehicle Technician', 'Data Modeler',
       'Sales Data Analyst', 'Cloud Data Engineer',
       'Finance Data Analyst', 'Research Scientist',
       'Data Analytics Manager', 'Data Specialist',
       'Data Operations Analyst', 'Director of Data Science',
       'Data Science Manager', 'Data Science Lead',
       'Machine Learning Scientist', 'AI Scientist',
       'Computer Vision Engineer', 'Data Science C

In [23]:
job_categories = {
    'data_science_and_ml': ['Data Scientist', 'Applied Scientist', 'ML Engineer', 'Machine Learning Engineer', 'Applied Machine Learning Engineer', 'Research Engineer', 'Machine Learning Developer', 'Machine Learning Scientist', 'AI Scientist', 'Machine Learning Software Engineer', 'Principal Data Scientist', 'Machine Learning Researcher', 'Decision Scientist', 'Machine Learning Research Engineer', 'Principal Machine Learning Engineer', 'Machine Learning Manager', 'Lead Machine Learning Engineer', 'Principal Data Analyst', 'Staff Data Scientist', 'Applied Machine Learning Scientist', 'Lead Data Scientist'],
    'data_engineering_and_architecture': ['AWS Data Architect', 'Data Engineer', 'Data Manager', 'Data Architect', 'Data Integration Specialist', 'ETL Developer', 'BI Data Analyst', 'Data Modeler', 'Cloud Data Engineer', 'Data Operations Analyst', 'ETL Engineer', 'Data DevOps Engineer', 'Data Quality Analyst', 'Big Data Engineer', 'BI Data Engineer', 'Cloud Database Engineer', 'Data Infrastructure Engineer', 'Software Data Engineer', 'Data Operations Engineer', 'Big Data Architect', 'Azure Data Engineer', 'Marketing Data Engineer', 'Data Management Specialist', 'Principal Data Architect', 'Cloud Data Architect', 'Principal Data Engineer', 'Machine Learning Infrastructure Engineer', 'MLOps Engineer', 'Analytics Engineer'],
    'data_analysis_and_visualization': ['Data Analyst', 'Product Data Analyst', 'Data Visualization Analyst', 'Sales Data Analyst', 'BI Developer', 'Business Data Analyst', 'Lead Data Analyst', 'BI Analyst', 'Insight Analyst', 'Marketing Data Analyst', 'Data Analytics Lead'],
    'data_management_and_strategy': ['Head of Data', 'Data Specialist', 'Director of Data Science', 'Data Science Manager', 'Data Science Lead', 'Data Strategist', 'Data Analytics Manager', 'Manager Data Management', 'Data Management Specialist'],
    'ai_development': ['Autonomous Vehicle Operations Lead', 'Autonomous Vehicle Technician', 'AI Developer', 'AI Programmer'],
    'computer_vision': ['Computer Vision Engineer', 'Computer Vision Software Engineer', '3D Computer Vision Researcher'],
    'nlp_engineer': ['NLP Engineer'],
    'business_intelligence': ['Business Intelligence Engineer', 'BI Data Analyst', 'BI Developer', 'BI Data Engineer', 'BI Analyst', 'Power BI Developer'],
    'research_and_science': ['Research Scientist', 'Applied Data Scientist', 'Data Science Consultant', 'Deep Learning Researcher', 'Deep Learning Engineer'],
    'other_data_related': ['Finance Data Analyst', 'Compliance Data Analyst', 'Data Science Tech Lead', 'Marketing Data Scientist', 'Product Data Scientist', 'Staff Data Analyst', 'Data Analytics Specialist', 'Data Analytics Engineer', 'Data Analytics Consultant', 'Data Management Specialist', 'Head of Data Science', 'Data Lead', 'Data Science Engineer', 'Insight Analyst', 'Data Science Engineer', 'Data Analytics Engineer', 'Data Analytics Consultant', 'Data Management Specialist', 'Data Scientist Lead', 'Principal Data Architect', 'Lead Data Engineer', 'Head of Machine Learning', 'Staff Data Scientist', 'Financial Data Analyst']
}

In [24]:
# check to make sure all jobs fall in a category

not_cat = []
for i in range(len(data.job_title)):
    title = data.job_title[i]
    categorized = False
    for key in job_categories.keys():
        cat = job_categories[key]
        if title in cat:
            categorized = True
    if not categorized:
        not_cat.append(title)

list(set(not_cat))

[]

In [25]:
# bin jobs based on category

binned_jobs = [None] * len(data.job_title)
for i in range(len(data.job_title)):
    title = data.job_title[i]
    categorized = False
    for key in job_categories.keys():
        cat = job_categories[key]
        if title in cat:
            binned_jobs[i] = key


data['job_title_binned'] = binned_jobs

In [26]:
len(data.job_title_binned.unique())

10

In [27]:
data.job_title_binned.value_counts()

data_science_and_ml                  1534
data_engineering_and_architecture    1474
data_analysis_and_visualization       692
data_management_and_strategy          146
research_and_science                  133
business_intelligence                  50
other_data_related                     49
computer_vision                        29
ai_development                         16
nlp_engineer                           11
Name: job_title_binned, dtype: int64

In [28]:
data.head()

,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size,job_title_binned
0,2023,MI,FT,AWS Data Architect,258000,USD,258000,US,100,US,L,data_engineering_and_architecture
1,2023,SE,FT,Data Scientist,225000,USD,225000,US,0,US,M,data_science_and_ml
2,2023,SE,FT,Data Scientist,156400,USD,156400,US,0,US,M,data_science_and_ml
3,2023,SE,FT,Data Engineer,190000,USD,190000,US,100,US,M,data_engineering_and_architecture
4,2023,SE,FT,Data Engineer,150000,USD,150000,US,100,US,M,data_engineering_and_architecture


In [29]:
data.to_csv('binned_jobs.csv', index=False) 